# Video Game Recommendation Engine

## Outline:
1. [Problem Statement](#Problem-Statement)
2. [Proposed Solution](#Proposed-Solution)
3. [How it integrates with my strategy for game dev.](#Strategy-Integration)
4. [The Data](#The-Data)
5. [Modelling](#Modelling)
6. [Accuracy and Iteration](#Accuracy-and-Iteration)
7. [Insights](#Insights)
8. [Appify it](#Appify-it)

## Problem Statement
Video games is a big business. If you're a developer, it's easy to get lost in the crowd, especially if you're not marketting to your best audience. Developing a game is a lengthy process, and if you're an indie dev it means years of time investment with potentially no payoff.

## Proposed Solution
What if we could use collaborative filtering to build a recommendation engine and size up the opportunity for potential new games with various genres or features? 

The steps will include: _Exploring the data, Building the Model, Evaluating, Iterating, and Extracting some Insights. Once these steps are complete, it can be deployed for broader use._

## Strategy Integration
I'm personally interested in this topic, since I'm bootstrapping my own project. It will have scifi and tactical rpg themes, so this will be especially useful for my own work.

## The Data
I'm utilizing the data collected and cleaned by NewbieIndieGameDev from Steam's api. Link: https://github.com/NewbieIndieGameDev/steam-insights#

It includes game data for genres, tags, and most importantly reviews.

First, I need to extract the data in an easy to use way. I want to practice with airflow, so I'll be doing a simple one-time pipeline.

In [2]:
!pip install pyarrow
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 94.1 MB/s eta 0:00:00:00:0100:01


In [3]:
!pwd

/notebooks/collaborative_filtering_practice


In [4]:
import pandas as pd
import zipfile
from pathlib import Path
import pyarrow.csv as pv
import pyarrow.parquet as pq
#from io import TextIOWrapper
import duckdb

path = Path("./data/steam-insights")

In [5]:
def zip_to_parquet(zip_file, path_dir=path):
    """
    Adapted with help of ChatGPT.
    """
    with zipfile.ZipFile(path_dir/zip_file, 'r') as z:
        for fname in z.namelist():
            if fname.endswith('.csv'):
                with z.open(fname) as f:
                    read_options = pv.ReadOptions(autogenerate_column_names=False)
                    parse_options = pv.ParseOptions(delimiter=',', quote_char='"', escape_char="\\", newlines_in_values=True)
                    convert_options = pv.ConvertOptions(strings_can_be_null=True)
                    try:
                        table = pv.read_csv(f,
                                            read_options=read_options,
                                            parse_options=parse_options,
                                            convert_options=convert_options
                                           )
                        parquet_path = path_dir / (Path(fname).stem + ".parquet")
                        pq.write_table(table, parquet_path)
                        
                    except Exception as e:
                        print(f"Failed to parse {fname}: {e}")
                        
#def inspect_zip(zip_file, path_dir=path):
#    df_list = []
#    with zipfile.ZipFile(path_dir/zip_file, 'r') as z:
#        for fname in z.namelist():
#            if fname.endswith('.csv'):
#                with z.open(fname) as f:
#                    with TextIOWrapper(f, encoding='utf-8', errors='replace') as wrapped_f:
#                        df = pd.read_csv(wrapped_f, on_bad_lines='warn', low_memory=False, delimiter=',', quotechar='"', escapechar="\\")
#                        df_list.append(df)
#    return(df_list)
#    

In [6]:
pq_list = [
    "descriptions.zip",
    "games.zip",
    "reviews.zip"
]

for p in pq_list:
    zip_to_parquet(p)

All right, I've got the data into parquet files; let's explore it a bit.

The benefit of parquet is that we can easily query the files as if they existed inside a relational database. Here's an example:

In [7]:
games_path = path / 'games.parquet'
sql = f"""
SELECT
    *
FROM '{games_path}'
LIMIT 100
"""

Then, we take the query and execute on it. The `.df()` method converts to a pandas dataframe.

In [8]:
con = duckdb.connect()
#Execute the query against the files and convert to pandas df
df = con.execute(sql).df()
display(df)

,app_id,name,release_date,is_free,price_overview,languages,type
0,10,Counter-Strike,2000-11-01,0,"{""final"": 819, ""initial"": 819, ""currency"": ""EU...","English<strong>*</strong>, French<strong>*</st...",game
1,20,Team Fortress Classic,1999-04-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game
2,30,Day of Defeat,2003-05-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Spain",game
3,40,Deathmatch Classic,2001-06-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game
4,50,Half-Life: Opposing Force,1999-11-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Korean",game
...,...,...,...,...,...,...,...
95,3190,King's Bounty Armored Princess Demo,2009-12-02,1,N,English,demo
96,3210,Painkiller Demo,2007-01-24,1,N,N,demo
97,3230,Genesis Rising,2007-08-06,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...",English,game
98,3260,Safecracker: The Ultimate Puzzle Adventure,2007-08-29,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, German, Italian, Spanish - Spain",game


Let's turn this into a object we can re-use.

In [9]:
def duckQuery(sql):
    con = duckdb.connect()
    df = con.execute(sql).df()
    return(df)

Let's do some investigations. I'm especially interested in looking at reviews per game, category, and tag.

In [10]:
table_path = path / 'reviews.parquet'
display(duckQuery(f"SELECT * FROM '{table_path}' LIMIT 10"))

,app_id,review_score,review_score_description,positive,negative,total,metacritic_score,reviews,recommendations,steamspy_user_score,steamspy_score_rank,steamspy_positive,steamspy_negative
0,10,9,Overwhelmingly Positive,235403,6207,241610,88,N,153259,0,N,235397,6207
1,20,8,Very Positive,7315,1094,8409,N,N,6268,0,N,7314,1092
2,30,8,Very Positive,6249,672,6921,79,N,4146,0,N,6246,672
3,40,8,Very Positive,2542,524,3066,N,N,2218,0,N,2541,525
4,50,9,Overwhelmingly Positive,22263,1111,23374,N,N,20144,0,N,22260,1112
5,60,8,Very Positive,4791,1013,5804,N,N,4145,0,N,4790,1013
6,70,9,Overwhelmingly Positive,129364,4652,134016,96,N,95770,0,N,129338,4660
7,80,8,Very Positive,23316,2220,25536,65,N,18898,0,N,23316,2218
8,130,8,Very Positive,15859,1330,17189,71,N,14830,0,N,15857,1330
9,219,8,Very Positive,132,10,142,96,N,N,0,N,73753,2522


In [17]:
# Query games
games_path = path / 'games.parquet'
display(duckQuery(f"SELECT * FROM '{games_path}' LIMIT 2"))
reviews_path = path / 'reviews.parquet'
display(duckQuery(f"SELECT * FROM '{table_path}' LIMIT 2"))
genres_path = path / 'genres.parquet'
display(duckQuery(f"SELECT * FROM '{genres_path}' LIMIT 2"))
categories_path = path / 'categories.parquet'
display(duckQuery(f"SELECT * FROM '{categories_path}' LIMIT 2"))

sql = f"""
    --Query games
    SELECT
        *
        
    FROM '{games_path}' as games
    --Join reviews
    LEFT JOIN '{reviews_path}' as r ON games.app_id = r.app_id
    --Join genres
    LEFT JOIN '{genres_path}' as g ON games.app_id = g.app_id
    --Join categories
    LEFT JOIN '{categories_path}' as c ON games.app_id = c.app_id
    
    LIMIT 10
"""


,app_id,name,release_date,is_free,price_overview,languages,type
0,10,Counter-Strike,2000-11-01,0,"{""final"": 819, ""initial"": 819, ""currency"": ""EU...","English<strong>*</strong>, French<strong>*</st...",game
1,20,Team Fortress Classic,1999-04-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game


,app_id,review_score,review_score_description,positive,negative,total,metacritic_score,reviews,recommendations,steamspy_user_score,steamspy_score_rank,steamspy_positive,steamspy_negative
0,10,9,Overwhelmingly Positive,235403,6207,241610,88,N,153259,0,N,235397,6207
1,20,8,Very Positive,7315,1094,8409,N,N,6268,0,N,7314,1092


,app_id,genre
0,10,Action
1,20,Action


,app_id,category
0,10,Family Sharing
1,10,Multi-player


In [16]:
ddf = duckQuery(sql)
display(ddf)

,app_id,name,release_date,is_free,price_overview,languages,type,app_id_1,review_score,review_score_description,...,reviews,recommendations,steamspy_user_score,steamspy_score_rank,steamspy_positive,steamspy_negative,app_id_2,genre,app_id_3,category
0,10,Counter-Strike,2000-11-01,0,"{""final"": 819, ""initial"": 819, ""currency"": ""EU...","English<strong>*</strong>, French<strong>*</st...",game,10,9,Overwhelmingly Positive,...,N,153259,0,N,235397,6207,10,Action,10,Valve Anti-Cheat enabled
1,20,Team Fortress Classic,1999-04-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game,20,8,Very Positive,...,N,6268,0,N,7314,1092,20,Action,20,Valve Anti-Cheat enabled
2,30,Day of Defeat,2003-05-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Spain",game,30,8,Very Positive,...,N,4146,0,N,6246,672,30,Action,30,Valve Anti-Cheat enabled
3,40,Deathmatch Classic,2001-06-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game,40,8,Very Positive,...,N,2218,0,N,2541,525,40,Action,40,Valve Anti-Cheat enabled
4,50,Half-Life: Opposing Force,1999-11-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Korean",game,50,9,Overwhelmingly Positive,...,N,20144,0,N,22260,1112,50,Action,50,Valve Anti-Cheat enabled
5,60,Ricochet,2000-11-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game,60,8,Very Positive,...,N,4145,0,N,4790,1013,60,Action,60,Valve Anti-Cheat enabled
6,70,Half-Life,1998-11-08,0,"{""final"": 11399, ""initial"": 11399, ""currency"":...","English<strong>*</strong>, French<strong>*</st...",game,70,9,Overwhelmingly Positive,...,N,95770,0,N,129338,4660,70,Action,70,Valve Anti-Cheat enabled
7,80,Counter-Strike: Condition Zero,2004-03-01,0,"{""final"": 819, ""initial"": 819, ""currency"": ""EU...","English, French, German, Italian, Spanish - Sp...",game,80,8,Very Positive,...,N,18898,0,N,23316,2218,80,Action,80,Valve Anti-Cheat enabled
8,130,Half-Life: Blue Shift,2001-06-01,0,"{""final"": 499, ""initial"": 499, ""currency"": ""EU...","English, French, German",game,130,8,Very Positive,...,N,14830,0,N,15857,1330,130,Action,130,Single-player
9,219,Half-Life 2: Demo,2004-12-01,1,N,English,demo,219,8,Very Positive,...,N,N,0,N,73753,2522,219,Action,219,Single-player


## Modelling

## Accuracy and Iteration

## Insights

## Appify it